In [1]:
import pandas as pd
import numpy as np

import time
import datetime

from sklearn.model_selection import KFold
from sklearn.model_selection._validation import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

1.Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [2]:
features = pd.read_csv('features/features.csv', index_col='match_id')
features.drop('duration radiant_win tower_status_radiant tower_status_dire barracks_status_radiant barracks_status_dire'.split(' '), inplace=True, axis=1)

In [3]:
features_test = pd.read_csv('features_test/features_test.csv', index_col='match_id')
#features_test.drop('duration radiant_win tower_status_radiant tower_status_dire barracks_status_radiant barracks_status_dire'.split(' '), inplace=True, axis=1)

2.Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [4]:
features[features[features.columns].notnull()].count()[lambda x: x < features.count().max()]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

first blood события, говорят нам о том, что первая кровь была сделана в первые пять минуть, а если нет пропуск. Думаю, это связанно со стилем игры проффесионалов, первые 5 минут идет прокачка. А низкое событие связано с тем, что чаще ферст блад происходил на мид лайне, а значит помощники отуствовали.

radiant_flying_courier_time небольшие значения, это связано с тактикой команд, можно предположить, что обычный курьер обходиться дешевле, а т.к. закупка чаще всего связана с таймингом комманды, то необходимости в нем мало.

3.Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. 

Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [5]:
features = features.fillna(0)
features_test = features_test.fillna(0)

4.Какой столбец содержит целевую переменную? Запишите его название.

In [6]:
'radiant_win' in features.columns.unique()

False

5.Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [7]:
X_train , Y_train = features.as_matrix(), pd.read_csv('features/features.csv', index_col='match_id')['radiant_win'].as_matrix()
X_test = features_test.as_matrix()
kfold = KFold(n_splits = 5, shuffle=True)

In [8]:
start_time = datetime.datetime.now()
scores=[]
for x in [1 if i==0 else i for i in range(0, 300, 10)]:
    clf = GradientBoostingClassifier(n_estimators=x, random_state=1)
    scores.append(np.array(cross_val_score(clf, X_train, Y_train,scoring='roc_auc', cv=kfold)))
scores=np.array(scores)
print ('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 3:16:22.847936


In [9]:
[np.mean(sc) for sc in scores]

[0.59687672529087021,
 0.66422167898711104,
 0.68218236517864506,
 0.68979370221600855,
 0.69386357185662828,
 0.69777085710319486,
 0.69950575950024008,
 0.70249135948595332,
 0.70413178631816875,
 0.70520258757832144,
 0.70651651017792938,
 0.70790106912846762,
 0.70828678281627644,
 0.70959208329424495,
 0.7098641290218255,
 0.71070917254802279,
 0.71083823271183333,
 0.71166964971411839,
 0.71295593604216723,
 0.71331876682397866,
 0.71409471959895165,
 0.71462730623588666,
 0.7147438966175097,
 0.71467949581822421,
 0.71539295442398509,
 0.71611106249880607,
 0.71711698648395505,
 0.71624675348726619,
 0.7167048613289223,
 0.71693639265756259]

In [18]:
rez = [1 if i==0 else i for i in range(0, 300, 10)][np.argmax([np.mean(sc) for sc in scores])]

Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

В целом, можно заметить, что алгоритм улучшается, с каждым разом метрика становится лучше. Но думаю, что это неверно, прирост качества незначителен.

Думаю, что я бы брал по 5 оценок, и нахоидл их отклонения. Вычислив минимальный порог эмперически, где качество практически не меняется, отберу нужное количество.

In [29]:
clf = GradientBoostingClassifier(n_estimators=rez, max_depth=4)
clf.fit(X_train, Y_train)
win = clf.predict_proba(X_test)

In [30]:
pd.DataFrame([[match, win[ind][1]] for ind, match in enumerate(features_test.index)], columns=['match_id', 'radiant_win']).to_csv('result.scv', index=False)

Score 0.70125, было при 30

Score 0.72559 при 260, и max_depth=4
